This Jupyter notebook allows you to run Python code interactively.

## Getting Started
1. Click the Select Kernel button at the top right.
2. Choose Python environments and select Python 3.11.9.

Run each section of the notebook by clicking the play button on the left side of the code cells.

## Learning Outcomes
We will focus on four key outcomes:

1. Understanding agents and prompt engineering with Prompty.
2. Utilizing Prompty tracing for debugging and observability.
3. Building and running Contoso Creative Writer.
4. Setting up automated evaluations with GitHub Actions.
Let’s start with the first one.

## 1. Understanding Agents and Prompt Engineering with Prompty
### What are AI Agents?
Contoso Creative Writer is an Agentic Application. In AI, an agent is a program designed to:

- Perceive its environment
- Make decisions
- Take actions to achieve specific goals

For Contoso Creative Writer, the goal is to help the marketing team at Contoso Outdoors write well-researched, product-specific articles. This is achieved by four agents, each handling a sub-goal.

Contoso Creative Writer is made up of 4 agents: 

<img src=“../images/agents.png” alt=“Agents in Contoso Creative Writer” width=“900” height=“380”>

### Building an Agent with Prompty and Azure OpenAI
We’ll focus on the Researcher Agent, which searches for relevant information online using Bing Search and Azure OpenAI with the GPT-3.5 Turbo model.

To see it in action, click the play button below. Try changing the instructions to research a topic you’re interested in, for example, “Can you find the best educational material for learning Python programming?”

In [ ]:
import sys
import os

# Add the path to sys.path
sys.path.append(os.path.abspath('../../src/api/agents/researcher'))

from researcher import research

instructions = "Can you find the best educational material for learning Python programming."

research(instructions=instructions)

#### How does the Researcher agent work? 

1. Navigate to the Folder: Click the file icon and go to `src/api/agents/researcher`.

2. Files Overview:

The researcher folder contains the following 3 files (click on each file to explore it as you read through this list):

- `functions.json`: Contains descriptions of tools (find_information, find_entities, find_news) the LLM can use. Each function includes a description of the parameters the LLM needs to generate as input, in this case search queries for the Bing Search API.

- `researcher.py`: Implements the functions described in functions.json and includes an execute function to handle instructions and outputs. The execute function passes user input to the researcher.prompty file.

- `researcher.prompty`: Standardizes prompts and their execution. It includes the prompt description, model details, and tool parameters from functions.json. The file also contains the base prompt sent to the LLM, including user instructions. 


#### An introduction to prompt engineering 

Prompt engineering is the process of designing and refining input prompts to guide generative AI models. With Prompty we can easily test and iterate by changing the user instructions or the base prompt to get better results from the LLM. 

To illustrate this let's change the instructions we send to the LLM to guide it to use a specific function. In our earlier example you likely had your results returned from the **find_information** function which returns its results in the `web` category, but you'll notice the `entities` and `news` categories are empty since their associated functions weren't called. The instructions we use can change this! 

1. Find entities: 

this function is used to find information about people or places. Let's ask the LLM to find information about Guido van Rossum the creator of Python. 


In [ ]:
instructions = "Can you do some research about the person Guido van Rossum?"

research(instructions=instructions)

2. Find news: 

This function is used to find news. Let's ask the LLM to find the latest news about Microsoft

In [ ]:
instructions = "Can you find the latest news about Microsoft? "

research(instructions=instructions)

Since we want to make sure the rest of the workshop works as expected we won't make any changes to the base prompt in `researcher.prompty`, but feel free to make edits it later to see how the results change! 

Let's now move onto the next Learing outcome. 

## 2. Utilizing Prompty tracing for debugging and observabilty

When running Applications driven by LLMs, sometimes things don't go as expected! It's important to have a way to debug your LLM workflow so you can see where things are working. 

Tracing helps you visualize the execution of your prompts and clearly see what inputs are being passed to the LLM. 

We'll use tracing to also get a better understanding of what's happening in our workflow by calling the functions that run the researcher, product and writer agents. 

Do the following to initiate tracing: 
- Run the cell below and then click the generated link (this might take a few minutes to appear)
- Once in the new tab click on the name `openai_chat` in the first row
- You should now be in the `converstions` tab. This tab contains the full prompt that will be sent to the LLM in Markdown format for readability. 
- Scroll to the top of the output and particularly look at the Research and Product Information sections. These sections contain the results generated by the Researcher and Product agents. If either of them were not producing the expected results we could easily spot it here! 
- Click on the `Raw Json` tab. This shows exactly what is sent to the LLM in the format it accepts, which is Json. 


In [ ]:
# Add the path to sys.path
sys.path.append(os.path.abspath('../../src/api'))

from promptflow.tracing import start_trace
from agents.researcher import researcher
from agents.product import product
from agents.writer import writer

research_context = "Can you find the latest camping trends and what folks are doing in the winter?"
product_context = "Can you use a selection of tents and sleeping bags as context?"
assignment_context = '''Write a fun and engaging article that includes the research and product information. 
    The article should be between 800 and 1000 words.
    Make sure to cite sources in the article as you mention the research not at the end.'''

research_result = researcher.research(research_context)
product_result = product.find_products(product_context)

start_trace()

writer_result = writer.write(
        research_context,
        research_result,
        product_context,
        product_result,
        assignment_context,
    )

processed_writer_result = writer.process(writer_result)
print(processed_writer_result)

Tracing is useful for deugging and observability both locally and in production. 

Let's now move on to the next section, building the Contoso Creative Writer Application!

## 3. Building and running Contoso Creative Writer 

Now that we understand how the application works it's time to build it. 

To complete these next two learning outcomes you'll need to use the terminal. 

If it’s not already visible, you can open it by clicking on the hamburger menu at the top left of the page, clicking Terminal and then selecting New Terminal.

Once your terminal is open, copy and past the following commands in the terminal and press enter after each one to run it. 

1. Starting the FastAPI server 

    Navigate to the src/api folder  with the following command 

    ```shell
    cd ./src/api
    ```

    Run the FastAPI webserver with the following command 

    ```shell
    fastapi dev main.py
    ```

Do not click open browser if prompted. 

Next you'll need to change the visibility of the API's 8000 and 5173 ports to public in the `PORTS` tab. You can do this by right clicking on the visibility section of the port, selecting port visibility and setting it to public. The ports tab should look like this:

<img src="../../images/ports.png" alt="Screenshot showing setting port-visibility" width="800px" />


2. Running the web app 

    Once you've completed the above steps. You'll need to open a **new terminal** and navigate to the web folder. you can open a new terminal by clicking on the hamburger menu at the top left of the page, clicking Terminal and then selecting New Terminal. 

    In the terminal run the following commands 

    ```shell
    cd ./src/web
    ```
    
    First install node packages:

    ```shell
    npm install
    ```

    Then run the web app with a local dev web server:
    
    ```shell
    npm run dev
    ```

Once you've run the above command you should see an `http://localhost:5173/` link in the terminal. Click this link or click the `open on browser` button that comes up as a Gitub notification in the bottom right corner of the screen. Select the `continue` button and that's it! 

You can now test out the app by clicking `Example` to fill out the example information and then clicking `Start Work` to get Contoso Creative Writer to generate an article. 

You can also see which agent steps are being carried out in what order by click on the small bug button at the bottom right of the Application. 


## 4. Setting up automated evaluations with Github Actions 

Contoso Creative Writer is set up to run a CI/CD pipeline, which stands for Continuous Integration and Continuous Deployment. This is a series of automated steps that streamline the process of delivering software.

- Continuous Integration (CI): This involves automatically integrating code changes from multiple contributors into a shared repository several times a day.
- Continuous Deployment (CD): This involves automatically deploying the integrated code to production environments. 

In this sample code the CI/CD pipeline includes the following: 
1. Build and Deploy: Automatically building and deploying the latest version of the code to production (This helps us confirm things are working as expected.)
2. Evaulations: Automatically sending example research, product and assignment instructions to Contoso Creative Writer and running evaulations on the produced article to see how fluent, grounded, relevant and x the final response was given the questions. 


To set up CI/CD with GitHub actions on your repository, open a new terminal and run the following command:

```shell
azd pipeline config
```

To see the results 